In [ ]:
# Dependencies managed via uv - see pyproject.toml
# This notebook requires: langchain-openai, python-dotenv

In [4]:
from langchain_openai import ChatOpenAI
import os
import asyncio
from typing import List

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_agent

# --- Configuration ---
from dotenv import load_dotenv
load_dotenv()

try:
    # A model with function/tool calling capabilities is required.
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    print(f"✅ Language model initialized: {llm.model_name}")
except Exception as e:
    print(f"🛑 Error initializing language model: {e}")
    llm = None


# --- Define a Tool ---
@tool
def search_information(query: str) -> str:
    """
    Provides factual information on a given topic. Use this tool to find answers to questions
    like 'What is the capital of France?' or 'What is the weather in London?'.
    """
    print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
    # Simulate a search tool with a dictionary of predefined results.
    simulated_results = {
        "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
        "capital of france": "The capital of France is Paris.",
        "population of earth": "The estimated population of Earth is around 8 billion people.",
        "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
        "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting."
    }
    result = simulated_results.get(query.lower(), simulated_results["default"])
    print(f"--- TOOL RESULT: {result} ---")
    return result

tools = [search_information]


# --- Create a Tool-Calling Agent ---
if llm:
    # In LangChain 1.0+, use create_agent from langchain.agents
    # This creates an agent that can use tools via the ReAct pattern
    agent_executor = create_agent(llm, tools)

    async def run_agent_with_tool(query: str):
        """Invokes the agent executor with a query and prints the final response."""
        print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
        try:
            # LangGraph agents use 'messages' as input
            response = await agent_executor.ainvoke(
                {"messages": [("human", query)]},
                {"recursion_limit": 10}
            )
            # Extract the final message from the response
            final_message = response["messages"][-1]
            print("\n--- ✅ Final Agent Response ---")
            print(final_message.content)
        except Exception as e:
            print(f"\n🛑 An error occurred during agent execution: {e}")

    async def main():
        """Runs all agent queries."""
        # Run queries sequentially to see clearer output
        await run_agent_with_tool("What is the capital of France?")
        await run_agent_with_tool("What's the weather like in London?")
        await run_agent_with_tool("Tell me something about dogs.")  # Should trigger the default tool response

    # In Jupyter, use await directly instead of asyncio.run()
    await main()

else:
    print("\nSkipping agent execution due to LLM initialization failure.")

✅ Language model initialized: gpt-4o

--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🛠️ Tool Called: search_information with query: 'What is the capital of France?' ---
--- TOOL RESULT: Simulated search result for 'What is the capital of France?': No specific information found, but the topic seems interesting. ---

--- ✅ Final Agent Response ---
The capital of France is Paris.

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🛠️ Tool Called: search_information with query: 'current weather in London' ---
--- TOOL RESULT: Simulated search result for 'current weather in London': No specific information found, but the topic seems interesting. ---

--- ✅ Final Agent Response ---
I couldn't retrieve the current weather information for London. You might want to check a reliable weather website or app for the latest updates.

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---

--- 🛠️ Tool Called: search_information with q